In [1]:
from transformers import pipeline
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

2023-03-06 06:57:56.297016: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 06:57:56.413370: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-06 06:57:56.985680: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib64:/opt/amazon/openmpi/lib64:/usr/local/cuda/efa/lib:/usr/local/cuda

In [3]:
# https://huggingface.co/docs/transformers/v4.26.1/en/main_classes/pipelines#transformers.ZeroShotClassificationPipeline
pipe = pipeline(
    model="facebook/bart-large-mnli",
    framework="pt",
    device=0,
    batch_size=10,
)

In [4]:
assert pipe.device.type == "cuda"

In [8]:
pipe("I have a problem with my iphone that needs to be resolved asap!",
     candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
     multi_label=True,
)

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.9987170696258545,
  0.9945850372314453,
  0.189903125166893,
  0.0007674109074287117,
  0.00038260716246441007]}

In [11]:
# all_labels = l2v_scores.columns.tolist()[4:]
# print(len(all_labels))
labels = ["Fracture", "Edema", "Cardiomegaly", "Pneumonia", "Atelectasis", "Pneumothorax", "Pleural Effusion"]

In [67]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files={
    "train": "/root/data/chex_train.csv",
    "val": "/root/data/chex_val.csv",
    "test": "/root/data/chex_test.csv",
}, split='val[:50%]')
dataset

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-50c8ec55ede3b71e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Unnamed: 0', 'Report Impression', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices', 'No Finding'],
    num_rows: 14615
})

In [37]:
dataset["val"][:3]

{'Unnamed: 0': [80343, 66645, 76374],
 'Report Impression': ['1.  No displaced fracture or traumatic malalignment of the left shoulder, humerus, elbow, forearm. Note: Radiographs are insensitive, and some fractures can be occult. If there is clinical concern for occult fracture, MRI has the highest sensitivity. Although CT may be helpful, nondisplaced fractures can be missed by CT exam.',
  '1.  Apparent retrocardiac opacity favored to be secondary to breast tissue.',
  '1.  No acute cardiopulmonary disease.'],
 'Enlarged Cardiomediastinum': [None, None, None],
 'Cardiomegaly': [None, None, None],
 'Lung Opacity': [None, 1.0, None],
 'Lung Lesion': [None, None, None],
 'Edema': [None, None, None],
 'Consolidation': [None, None, None],
 'Pneumonia': [None, None, None],
 'Atelectasis': [None, None, None],
 'Pneumothorax': [None, None, None],
 'Pleural Effusion': [None, None, None],
 'Pleural Other': [None, None, None],
 'Fracture': [-1.0, None, None],
 'Support Devices': [None, None, Non

In [69]:
key_dataset = KeyDataset(dataset, "Report Impression")
pred_all = pipe(
    sequences=key_dataset,
    candidate_labels=labels,
    multi_label=True,
)
print(len(pred_all))

results = []
for out in tqdm(pred_all, total=len(key_dataset)):
    results.append(out)
print(len(results))

1462


  0%|          | 0/14615 [00:00<?, ?it/s]

In [70]:
import pickle
pickle.dump(results, open('bart_3_6_2023_val_50pct.pkl', 'wb'))

In [61]:
from torch.utils.data import Dataset as TorchDataset

class MyDataset(TorchDataset):
    kdv = KeyDataset(dataset["val"], "Report Impression")
    def __len__(self):
        return 5

    def __getitem__(self, i):
        return "This is a test"
        return kdv[i]

pred3 = pipe(
    MyDataset(),
    candidate_labels=labels,
    multi_label=True,
)

results = []
i = 0
for out in pred3:
    results.append(out)
print(results)

KeyboardInterrupt: 

In [62]:
len(pred3)

1

In [18]:
dataset = Dataset.from_pandas(chexbert_results[["Report Impression"]][:20])

pred_all = pipe(
    sequences=KeyDataset(dataset, "Report Impression"),
    candidate_labels=labels,
    multi_label=True,
    batch_size=10,
)
print(len(pred_all))

results = []
i = 0
for out in pred_all:
    results.append(out)
#     print(out)
#     i += 1
#     if i == 3:
#         break
print(results)

2
[{'sequence': '1.  Interval increase in perihilar opacities and interstitial markings, suggestive of mild pulmonary edema. No new focal consolidation.', 'labels': ['Edema', 'Pneumonia', 'Pleural Effusion', 'Pneumothorax', 'Fracture', 'Atelectasis', 'Cardiomegaly'], 'scores': [0.9291210174560547, 0.5246122479438782, 0.18574288487434387, 0.11820070445537567, 0.009015855379402637, 0.005224664695560932, 0.002340974286198616]}, {'sequence': '1.  No acute intracranial abnormality. 2.  Mild right forehead scalp fat stranding, correlate with exam for possible mild soft tissue contusion versus cutaneous lesion. No fracture of the skull or imaged facial bones.', 'labels': ['Atelectasis', 'Fracture', 'Edema', 'Pleural Effusion', 'Cardiomegaly', 'Pneumothorax', 'Pneumonia'], 'scores': [0.1975606232881546, 0.1781543493270874, 0.10632673650979996, 0.0969989001750946, 0.009535315446555614, 0.0010840531904250383, 0.0007761431625112891]}, {'sequence': '1.  Multiple loops of dilated small bowel with f

In [24]:
dataset = Dataset.from_pandas(chexbert_results[["Report Impression"]][:64])
print("len dataset", len(dataset))

for batch_size in [1, 4, 16, 64]:
    pred_all = pipe(
        sequences=KeyDataset(dataset, "Report Impression"),
        candidate_labels=labels,
        multi_label=True,
        batch_size=batch_size,
    )
    print("pipe len:", len(pred_all))

    results = []
    i = 0
    for out in pred_all:
        results.append(out)
        i += 1
        print(i)

#     print(batch_size, len(results))

len dataset 64
pipe len: 64
1


/opt/conda/envs/l2v/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2
3
4
5
6
7
8
9


/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 65 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 66 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 67 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10

10


...

/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 141 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 142 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 143 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3

21


/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 152 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 153 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3.10/site-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f02481d6dd0> was reported to be 64 (when accessing len(dataloader)), but 154 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/l2v/lib/python3

22


KeyboardInterrupt: 

In [ ]:
import pickle
dict_a = {'A':0, 'B':1, 'C':2}
pickle.dump(results, open('hug_results.pkl', 'wb'))

In [13]:
results[0]

{'sequence': '1.  Interval increase in perihilar opacities and interstitial markings, suggestive of mild pulmonary edema. No new focal consolidation.',
 'labels': ['Edema',
  'Pneumonia',
  'Pleural Effusion',
  'Pneumothorax',
  'Fracture',
  'Atelectasis',
  'Cardiomegaly'],
 'scores': [0.9291203022003174,
  0.5246118307113647,
  0.18574422597885132,
  0.11820053309202194,
  0.009015824645757675,
  0.005224645137786865,
  0.002340991050004959]}

In [ ]:
"""
compare against chexbert, assuming chexbert is truth
"""
threshold=0.75
def to_pandas(results_list):
    labels = ["Fracture", "Edema", "Cardiomegaly", "Pneumonia", "Atelectasis", "Pneumothorax", "Pleural Effusion"]
#     labels = ['urgent', 'phone', 'computer', 'not urgent', 'tablet']
    schema = {col_name: pd.Series(dtype="float64") for col_name in labels}
    schema["Impression"] = pd.Series(dtype="object")
    rdf = pd.DataFrame(schema)
    for result in results:
        row = {label: score for label, score in zip(result["labels"], result["scores"])}
        row["Impression"] = result["sequence"]
        rdf = rdf.append(row, ignore_index=True)
    #     rdf = pd.concat([rdf, pd.DataFrame(row)])
    return rdf

results_df = to_pandas(results)

In [ ]:
def naive_bayes(y_pred, y_true, col_name):
    X = y_pred[[col_name]]
    y_true = y_true[col_name]
    y = y_true.mask(y_true != 1, 0)

    nb = GaussianNB()
    nb.fit(X, y)
    return nb

def evaluate(pred_df, true_df, col_name):
    """
    compare results against chexbert ground truth
    """
    nb = naive_bayes(pred_df, true_df, col_name)
    
    y_pred = nb.predict(X)
    print(label)
    print(metrics.classification_report(y, y_pred))
    fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC", auc)
    return nb
    
def compare_hist(y_pred, y_true, scale="", title=""):
    assert y_pred.shape == y_true.shape
    df = pd.DataFrame({ 'y_pred': y_pred, 'y_true': y_true })
    
    true_scores = df[df["y_true"] == 1]["y_pred"]
    false_scores = df[df["y_true"] != 1]["y_pred"]
#     print(true)
    bins = np.linspace(-1, 1, 100)

    plt.hist(true_scores, bins, alpha=0.5, label='true_scores')
    plt.hist(false_scores, bins, alpha=0.5, label='false_scores')
    plt.legend(loc='upper right')
    if scale == 'log':
        plt.yscale('log')
    plt.title(title)
    plt.show()

In [ ]:
compare_hist(results_df["Pneumonia"], chex_df["Pneumonia"], title="Pneumonia")

In [ ]:
for label in labels:
    evaluate(results_df, chex_df, label)